In [4]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import importlib
import os
import nn.classifier
import nn.unet as unet
import img.dataset as ds
from img import transformer
import color_classes as cc
    
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

from nn.train_callbacks import TensorboardVisualizerCallback, TensorboardLoggerCallback, ModelSaverCallback
from nn.test_callbacks import PredictionsSaverCallback
import helpers


In [5]:
importlib.reload(nn.unet)
importlib.reload(cc)
importlib.reload(transformer)
importlib.reload(ds)
importlib.reload(nn.classifier)
importlib.reload(nn.test_callbacks)

<module 'nn.test_callbacks' from '/home/ditschuk/kitcar/kitcar-gazebo-generation/segmentation/nn/test_callbacks.py'>

In [6]:
# Hyperparameters
    img_resize = (320, 160)
    batch_size = 1
    epochs = 3
    threshold = 0.5
    validation_size = 0.2
    sample_size = None  # Put None to work on full dataset

In [7]:
data_folder = os.environ.get('KITCAR_GAZEBO_SIM_PATH') + '/datasets/segmentation/'
train_folder = data_folder + 'train'
test_folder = data_folder + 'test'
labels_folder = data_folder + 'train_masks'

In [8]:
ex_img_name = os.listdir(train_folder)[100]
ex_img = cv2.imread(os.path.join(train_folder,ex_img_name))
ex_label_img = cv2.imread(os.path.join(labels_folder,ex_img_name))

In [9]:
net = unet.UNet1024((3, 160,320))
classifier = nn.classifier.CarvanaClassifier(net, epochs)

In [10]:
train_ds = ds.SegmentationImageDataset(data_folder)


In [11]:
from img.dataset import SegmentationImageDataset
from multiprocessing import cpu_count
threads = cpu_count()
use_cuda=False

In [12]:
train_loader = DataLoader(train_ds, batch_size,
                              sampler=RandomSampler(train_ds),
                              num_workers=threads,
                              pin_memory=use_cuda)

In [13]:
valid_ds = ds.SegmentationImageDataset(data_folder, train_folder='validate')
valid_loader = DataLoader(valid_ds, batch_size,
                              sampler=SequentialSampler(valid_ds),
                              num_workers=threads,
                              pin_memory=use_cuda)

In [14]:
print("Training on {} samples and validating on {} samples "
          .format(len(train_loader.dataset), len(valid_loader.dataset)))


Training on 319 samples and validating on 16 samples 


In [15]:
tb_viz_cb = TensorboardVisualizerCallback(os.path.join(data_folder, 'logs/tb_viz'))
tb_logs_cb = TensorboardLoggerCallback(os.path.join(data_folder, 'logs/tb_logs'))
model_saver_cb = ModelSaverCallback(os.path.join(data_folder, 'output/models/model_' +
                                                     helpers.get_model_timestamp()), verbose=True)
# Testing callbacks
pred_saver_cb = PredictionsSaverCallback(os.path.join(data_folder, 'output/submit.csv.gz'),
                                             [1280,650], threshold)

In [13]:
classifier.train(train_loader, valid_loader, epochs)


Epochs 1/3:   0%|          | 0/319 [?]/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:2390: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning

train_loss = 0.492840, train_acc = 0.929337
val_loss   = 0.095230, val_acc   = 0.983684
Time elapsed = 401s


Epochs 3/3:   0%|          | 0/319 [?]                     

train_loss = 0.053225, train_acc = 0.989890
val_loss   = 0.040612, val_acc   = 0.987381
Time elapsed = 396s


Epochs 3/3: 100%|██████████| 319/319 [00:00, loss=0.01393, dice_coeff=0.99709]
                                                           

train_loss = 0.025237, train_acc = 0.992932
val_loss   = 0.024142, val_acc   = 0.991942
Time elapsed = 396s


In [14]:
test_ds = ds.SegmentationImageDataset(data_folder, train_folder='validate')
test_loader = DataLoader(test_ds, batch_size,
                             sampler=SequentialSampler(test_ds),
                             num_workers=threads,
                             pin_memory=use_cuda)

In [3]:
ex_img_name = os.listdir(test_folder)[2]
ex_img = cv2.imread(os.path.join(test_folder,ex_img_name))
ex_label_img = cv2.imread(os.path.join(labels_folder,ex_img_name))
cropped = transformer.center_cropping_resize(ex_img,[320,160])
t = torch.from_numpy(cropped.transpose(2,0,1)).float().unsqueeze(0)
res = net(t)[0][1].detach()
plt.subplot(221)
plt.imshow(cropped)
plt.title('Image')
plt.subplot(222)
plt.imshow(ex_label_img)
plt.title('Label')
plt.show()

plt.imshow(res)
plt.title('Result')
plt.show()

NameError: name 'os' is not defined